In [1]:
import pandas as pd

In [30]:
import re


def extract_end_to_end_delay_mean(filename):
    end_to_end_delay_means = []
    end_to_end_jitter_means = []
    package_sum = 0
    package_receive_sum = 0

    with open(filename, 'r') as file:
        file_str = file.read()
            # 使用正则表达式找到包含 endToEndDelay:mean 的行
        delay_matchs = re.findall(r'endToEndDelay:mean (\d+\.\d+)', file_str)
        jitter_matchs = re.findall(r'endToEndJitter:mean (\d+\.\d+)', file_str)
        package_nums = re.findall(r'sendCount:count (\d+)', file_str)
        package_receives = re.findall(r'receiveCount:count (\d+)', file_str)

        if delay_matchs:
            end_to_end_delay_means = [float(match) for match in delay_matchs]
        if jitter_matchs:
            end_to_end_jitter_means = [float(match) for match in jitter_matchs]
        if package_nums:
            package_sum = sum([int(match) for match in package_nums])
        if package_receives:
            package_receive_sum = sum([int(match) for match in package_receives])
            
        packet_length = re.search(r'packetLength\s+(\d+byte)', file_str)
        sendIaTime = re.search(r'sendIaTime (exponential\((.*?)\))', file_str)
            
        if packet_length and sendIaTime:
            packet_length = packet_length.group(1)
            sendIaTime = sendIaTime.group(1)
        else:
            print(packet_length)
            print(sendIaTime)
            raise Exception("未统计packet_length和sendIaTime")

    return packet_length,sendIaTime, sum(end_to_end_delay_means) / len(end_to_end_delay_means), sum(end_to_end_jitter_means) / len(
        end_to_end_jitter_means), (package_sum - package_receive_sum) / package_sum

In [31]:
packet_length,sendIaTime, end_to_end_delay_means, end_to_end_jitter_means, plr = extract_end_to_end_delay_mean("results/0/General-#0.sca")
plr

0.24258498064846648

In [33]:
def result_statistic(net_id_list):
    columns = ['net_id','packet_length','sendIaTime', 'end_to_end_delay', 'end_to_end_jitter', 'package_loss_rate']
    df = pd.DataFrame(columns=columns,index=None)
    for i in net_id_list:
        data_path = f"results/{i}/General-#0.sca"
        packet_length,sendIaTime, end_to_end_delay_means, end_to_end_jitter_means, plr = extract_end_to_end_delay_mean(data_path)
        entry = {
            'net_id': i,
            'packet_length':packet_length,
            'sendIaTime':sendIaTime,
            'end_to_end_delay': end_to_end_delay_means,
            'end_to_end_jitter': end_to_end_jitter_means,
            'package_loss_rate': plr
        }
        df.loc[len(df)] = entry
    return df
df  = result_statistic(range(10))
df.head(5)

,net_id,packet_length,sendIaTime,end_to_end_delay,end_to_end_jitter,package_loss_rate
0,0,4096byte,exponential(100ms),956.049396,2.584084,0.242585
1,1,4096byte,exponential(1s),1.007156,3.076476,0.000008
2,2,4096byte,exponential(1s),492.758720,2.894900,0.020339
3,3,4096byte,exponential(1s),1547.751103,4.722917,0.077650
4,4,4096byte,exponential(1s),3153.400017,2.411540,0.155108


In [37]:
import os


def collect_integer_named_folders(directory):
    # 正则表达式匹配整数命名的文件夹
    pattern = re.compile(r'^\d+$')
    
    # 收集文件夹名称列表
    integer_named_folders = []
    
    # 遍历目录中的所有文件和文件夹
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path) and pattern.match(item):
            integer_named_folders.append(item)
    integer_named_folders.sort()
    return integer_named_folders

# 指定要搜索的目录路径
directory_path = 'results'

# 获取所有以整数命名的文件夹列表
integer_folders = collect_integer_named_folders(directory_path)
integer_folders

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
import pandas as pd
def analyse_datalog_file(file_path):
    file_path = file_path
    # 初始化数据存储
    data = []
    
    # 读取文件内容并解析
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # 解析每一行并组织数据
    send_pkg_dict = {}
    node_pair_map = {}
    for line in lines:
        line = line.strip()
        if line.startswith("src:"):
            parts = line.split(", ")
            src = int(parts[0].split(": ")[1])
            dst = int(parts[1].split(": ")[1])
            if not (src,dst) in node_pair_map:
                node_pair_map[(src,dst)] = {}
                
            if 'sendPkgCount' in parts[2]:
                sendPkgCount = int(parts[2].split(": ")[1])
                node_pair_map[(src,dst)]['sendPkgCount'] = sendPkgCount
                
            else:
                avgDelay = float(parts[2].split(": ")[1])
                node_pair_map[(src,dst)]['avgDelay'] = avgDelay
                avgJitter = float(parts[3].split(": ")[1])
                node_pair_map[(src,dst)]['avgJitter'] = avgJitter
                
                pkgReceiveCount = int(parts[4].split(": ")[1])
                node_pair_map[(src,dst)]['pkgReceiveCount'] = pkgReceiveCount
                
    for k, v in node_pair_map.items():   
        src, dst = k
        
        if 'pkgReceiveCount' in v:
            data.append({
                'src': src,
                'dst': dst,
                'avgDelay': v['avgDelay'] if 'avgDelay' in v else -1,
                'avgJitter':v['avgJitter'] if 'avgDelay' in v else -1,
                'pkgLossRate': v['pkgReceiveCount']/v['sendPkgCount'] if 'pkgReceiveCount' in v else -1
            })

    # 创建 DataFrame
    df = pd.DataFrame(data, columns=['src', 'dst', 'avgDelay', 'avgJitter','pkgLossRate'])
    # 打印 DataFrame
    return df

analyse_datalog_file('logs/87/pkg_log.txt')

,src,dst,avgDelay,avgJitter,pkgLossRate
0,2,36,6.17536,1.053850,0.986957
1,3,38,5.73164,1.160690,1.000000
2,4,38,7.07240,1.054160,1.000000
3,5,36,2.70167,0.059093,1.000000
4,6,39,3.77342,0.994522,1.000000
5,7,37,1.85727,0.028162,0.992000
6,9,36,3.21745,0.039634,0.995192


In [13]:
import re
import os
def collect_integer_named_folders(directory):
    # 正则表达式匹配整数命名的文件夹
    pattern = re.compile(r'^\d+$')

    # 收集文件夹名称列表
    integer_named_folders = []

    # 遍历目录中的所有文件和文件夹
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path) and pattern.match(item):
            integer_named_folders.append(item)
    integer_named_folders.sort()
    return integer_named_folders

def analyse_datalog_dir(data_log_dir, output_dir):
    # 每个网络单独一张表
    net_id_list = collect_integer_named_folders(data_log_dir)
    for i in net_id_list:
        data_log_path =f'logs/{i}/pkg_log.txt'
        df = analyse_datalog_file(data_log_path)
        df.to_csv(os.path.join(output_dir,f'net{i}_path_qos.csv'), index=False)

    print("finished")

analyse_datalog_dir('./logs', './net_path_df')

finished


In [1]:
from network_sim_server import analyse_datalog_file
res = analyse_datalog_file('logs/3/pkg_log.txt')
res

KeyError: 'avgDelay'